# packages

In [1]:
import os
import sys
import time
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
sys.path.append('/Users/yantingting/PycharmProjects/modeling/')
import database_conncet as data_con
import data_processing as dp
import summary_statistics as ss
from data_io_utils import *
from matplotlib.pyplot import MultipleLocator
from jinja2 import Template

In [2]:
file_path = '/Users/yantingting/Seafile/风控/模型/10 印尼/用户画像/定位优质客群共性/data/'
if not os.path.exists(file_path):
    os.makedirs(file_path)

# 取数

In [ ]:
query0 = '''
create table ytt_yhhx0305 as select t1.customer_id as customer_id 
from rt_t_gocash_core_loan t1 
left join rt_t_gocash_core_customer t2 
on t1.customer_id = t2.id 
where date(t2.create_time)>='2019-01-01' and date(t2.create_time)<='2020-03-04' and date(t1.apply_time)<='2020-03-05'
group by t1.customer_id
having count(distinct case when effective_date != '1970-01-01' then loan_no else null end )>1
'''

In [ ]:
time1 = time.time()
df0 =  data_con.database_gn(query0)
time2 = time.time()
print('run_time: ', time2 - time1)

In [ ]:
query1= '''
select t1.customer_id,
min(date(apply_time)) as first_apply_date,
max(date(apply_time)) as last_apply_date,
max(date_part('days',date('2020-03-05') - apply_time) ) first_apply_now,
min(date_part('days',date('2020-03-05') - apply_time)) last_apply_now,
count(distinct case when loan_status = 'DENIED' then id end ) as cnt_denied,
count(distinct case when loan_status = 'RESCIND' then id end ) as cnt_rescind,
count(distinct case when loan_status = 'APPROVED' or loan_status = 'FUNDED' or loan_status = 'CREATED' or loan_status = 'COLLECTION' then id end) as new_flag,
count(distinct case when loan_status = 'ADVANCE_PAIDOFF'  then id end) as cnt_advance,
count(distinct case when loan_status = 'PAIDOFF'  then id end) as cnt_paidoff,
count(distinct case when loan_status = 'COLLECTION'  then id end) as cnt_collection,
count(distinct case when loan_status = 'COLLECTION_PAIDOFF'  then id end) as cnt_collectionpaid,
count(distinct loan_no ) as cnt_apply,
count(distinct case when effective_date != '1970-01-01' then loan_no else null end ) as cnt_loan,
sum(case when effective_date != '1970-01-01' then extend_times else 0 end) cnt_extend,
max(case when effective_date != '1970-01-01' then extend_times else 0 end) max_extend,
min(case when effective_date !='1970-01-01' and effective_date is not null then effective_date else null end ) as first_effective_date,
max(case when effective_date !='1970-01-01' and effective_date is not null then effective_date else null end ) as last_effective_date,
max(case when effective_date !='1970-01-01' and effective_date is not null then date('2020-03-05') - effective_date else null end) first_effective_now,
min(case when effective_date !='1970-01-01' and effective_date is not null then date('2020-03-05') - effective_date else null end) last_effective_now,
count(distinct case when date('2020-03-05') - effective_date<=90 and  effective_date!= '1970-01-01' then loan_no else null end ) cnt_loan_90d,
count(distinct case when date('2020-03-05') - effective_date<=180 and  effective_date!= '1970-01-01' then loan_no else null end ) cnt_loan_180d
from ytt_yhhx0305 t1 
left join dw_gocash_go_cash_loan_gocash_core_loan t2 
on t1.customer_id = t2.customer_id
group by 1;
'''

In [ ]:
df1 =  data_con.database_gn(query1)
save_data_to_pickle(df1, file_path, '历史行为.pkl')
df1.shape
df1.head()

In [ ]:
query2 = '''
select
	u.customer_id as customer_id,
	u.max_late_days,
	u.total_order,
	u.total_loan,
	u.rucui_loan,
	u.rucui_order,
	v.chucui_loan,
	v.chucui_order,
	v.chucui_days,
	v.max_chucui_days
from
	(
	select
		a.customer_id as customer_id,
		max(late_day) as max_late_days,
		count(*) as total_order,
		count(distinct a.id) as total_loan,
		count(distinct case when b.order_status is not null and b.order_status <> 'PAIDOFF' then a.id else null end) as rucui_loan,
		count(distinct case when b.order_status is not null and b.order_status <> 'PAIDOFF' then b.id else null end) as rucui_order
	from
		(
		select
			id,
			customer_id,
			approved_principal,
			approved_period,
			effective_date,
			return_flag
		from
			public.dw_gocash_go_cash_loan_gocash_core_loan
		)a
	left join (
		select
			id,
			order_id,
			order_status,
			late_day,
			row_number() over(partition by order_id
		order by
			create_time) as rk
		from
			public.dw_gocash_gocash_collection_col_case)b on
		a.id = b.order_id
	group by
		a.customer_id)u
left join (
	select
		a.customer_id as customer_id,
		max(late_days) as max_chucui_days,
		sum(late_days) as chucui_days,
		count(distinct case when late_fee>0 then a.id else null end) as chucui_loan,
		count(distinct case when late_fee>0 then b.id else null end) as chucui_order
	from
		(
		select
			id,
			customer_id,
			approved_principal,
			approved_period,
			effective_date,
			return_flag
		from
			public.dw_gocash_go_cash_loan_gocash_core_loan
		)a
	left join (
		select
			id,
			loan_id,
			late_days,
			late_fee,
			row_number() over(partition by loan_id
		order by
			create_time) as rk
		from
			public.dw_gocash_go_cash_loan_gocash_core_loan_pay_flow
		where
			status = 'SUCCESS'
			and late_fee>0)b on
		a.id = b.loan_id
	group by
		a.customer_id) v 
		on	v.customer_id = u.customer_id
'''

In [ ]:
df2 = data_con.database_gn(query2)
save_data_to_pickle(df2, file_path, '催收情况.pkl')
df2.shape
df2.head()

In [ ]:
query3 = '''
select t1.customer_id as customer_id,
date(t2.create_time) as reg_date,
t2.marital_status,
t2.religion,
t2.education,
t2.media_source,
substring(id_card_no, 1,2) as provincecode , 
substring(id_card_no, 3,2) as citycode ,
substring(id_card_no, 5,2) as districtcode , 
case when substring(id_card_no, 7,2)::int > 40 then 'female' else 'male' end as gender ,--female生日 = 1-31 + 40; male生日=1-31
case when substring(id_card_no, 11,2)::int >substring(cast(update_time as varchar),3,2)::int 
     then 100 +substring(cast(update_time as varchar),3,2)::int - substring(id_card_no, 11,2)::int
     else substring(cast(update_time as varchar),3,2)::int - substring(id_card_no, 11,2)::int  end as age
from ytt_yhhx0305 t1 
left join rt_t_gocash_core_customer t2 
on t1.customer_id = t2.id
'''

In [ ]:
df3 = data_con.database_gn(query3)
save_data_to_pickle(df3, file_path, '基本信息.pkl')
df3.shape
df3.head()

In [ ]:
query4 = '''
select customer_id
, device_info::json ->> 'brand' as brand
    , device_info::json ->> 'heightPixels'  as heightPixels
    , device_info::json ->> 'widthPixels'  as widthPixels
    , device_info::json ->> 'manufacturer'  as manufacturer
    , device_info::json ->> 'Model'::varchar  as model
    , device_info::json ->> 'version'  as version
from(select t1.customer_id,t2.device_info,
row_number()over(partition by t2.customer_id order by t2.create_time desc) as rn
from ytt_yhhx0305 t1 
left join (select customer_id,create_time,
case when substring(device_info,1,1)='"' then substring(device_info,2,length(device_info)-2) else device_info end as device_info
from gocash_loan_risk_program_baseinfo
WHERE device_info<>'') t2 
on t1.customer_id::varchar = t2.customer_id)t
where rn = 1;
'''

In [ ]:
df4 = data_con.database_gn(query4)
save_data_to_pickle(df4, file_path, '设备信息.pkl')
df4.shape
df4.head()

In [ ]:
query5 = '''
select customer_id,
packages_cleaned_2
from(select t1.customer_id,t2.packages_cleaned_2,
row_number()over(partition by t2.customer_id order by t2.create_time desc) as rn
from ytt_yhhx0305 t1 
left join gocash_loan_risk_program_packages t2 
on t1.customer_id::varchar = t2.customer_id)t
where rn = 1;
'''

In [ ]:
df5 = data_con.database_gn(query5)
save_data_to_pickle(df5, file_path, 'APP信息.pkl')
df5.shape
df5.head()

In [4]:
query6 = '''
select *,
case when last_paid_off_time !='1970-01-01' and last_paid_off_time is not null then date('2020-02-18') - date(last_paid_off_time) else null end last_paid_now
from (
select t1.customer_id as customer_id,
sum(t2.extend_fee+t2.late_fee+t2.approved_principal-t2.approved_amount ) as profit,
max(t2.last_paid_off_time) as last_paid_off_time
from 
(select * from ytt_yhhx0305)t1
left join 
(select a1.id as loan_id,
a1.customer_id as customer_id,
a1.effective_date,
a1.approved_principal,
a1.approved_amount,
a2.last_paid_off_time,
case when a2.extend_fee is not null then a2.extend_fee else 0 end as extend_fee,
case when a2.late_fee is not null then a2.late_fee else 0 end as late_fee,
case when a2.principal is not null then a2.principal else 0 end as principal_repay
from 
(select * from dw_gocash_go_cash_loan_gocash_core_loan
where effective_date is not null) a1 
left join 
(select loan_id,
sum(case when extend_fee is not null then extend_fee else 0 end)  as extend_fee,
sum(case when late_fee is not null then late_fee else 0 end)  as late_fee,
sum(case when principal is not null then principal else 0 end)  as principal,
max(case when date(paid_off_time) !='1970-01-01' then paid_off_time else null end) as last_paid_off_time
from dw_gocash_go_cash_loan_gocash_core_loan_pay_flow
where status = 'SUCCESS'
and date(create_time)<='2020-02-18'
group by loan_id)a2 
on a1.id = a2.loan_id) t2 
on t1.customer_id = t2.customer_id
group by t1.customer_id) t;
'''

In [6]:
df6 = data_con.database_gn(query6)
df6['profit_rmb'] = df6['profit']/2000
save_data_to_pickle(df6, file_path, '盈利性.pkl')
df6.shape
df6.head()

Successful connection


(143044, 5)

,customer_id,profit,last_paid_off_time,last_paid_now,profit_rmb
0,145866999474814976,1770500.0,2019-06-28 22:01:33,235.0,885.25
1,156498181539368960,8742200.0,2020-02-15 08:36:53,3.0,4371.10
2,159786071656632320,2237900.0,2019-08-12 13:13:46,190.0,1118.95
3,161232718844534784,2102500.0,2019-09-28 19:43:44,143.0,1051.25
4,161268255131312128,1048000.0,2019-07-04 18:34:00,229.0,524.00


# 数据合并和变量衍生

## other

In [3]:
df1 = load_data_from_pickle(file_path, '历史行为.pkl')
# df1.head()
df1['is_new'] = df1['first_effective_now'].apply(lambda x: 1 if x <=50 else 0)
remain_col1 = ['customer_id','cnt_loan','cnt_extend','max_extend','cnt_advance','last_apply_now','is_new']
df_history = df1[remain_col1].copy()
df_history.shape
df_history.head()

(143044, 7)

,customer_id,cnt_loan,cnt_extend,max_extend,cnt_advance,last_apply_now,is_new
0,145866999474814976,4,3,3,1,295.0,0
1,161268255131312128,3,1,1,1,259.0,0
2,164066402572308480,6,2,1,3,235.0,0
3,164726759007584256,3,0,0,1,316.0,0
4,170593115028455424,3,3,3,0,219.0,0


In [4]:
df1['is_new'].value_counts()

0    115915
1     27129
Name: is_new, dtype: int64

In [9]:
df2 = load_data_from_pickle(file_path, '催收情况.pkl')
df2['dpd7+'] = df2['max_late_days'].apply(lambda x: 1 if x>=7 else 0)
remain_col2 = ['customer_id','max_late_days','rucui_loan','rucui_order','dpd7+']
df_collection = df2[remain_col2].copy()
df_collection.head()

,customer_id,max_late_days,rucui_loan,rucui_order,dpd7+
0,141828449351630848,796.0,1,1,1
1,141850255533932544,796.0,1,1,1
2,142717683927228416,NaN,0,0,0
3,142727694476779520,NaN,0,0,0
4,142738060309041152,725.0,1,1,1


In [10]:
df3 = load_data_from_pickle(file_path, '基本信息.pkl')
df_baseinfo = df3.copy()
df_baseinfo.shape
df_baseinfo.head()

(143044, 11)

,customer_id,reg_date,marital_status,religion,education,media_source,provincecode,citycode,districtcode,gender,age
0,363324722938687488,2019-07-31,MARRIED,ISLAM,SENIOR_HIGH_SCHOOL,organic,32,73,14,male,42
1,312257284621246464,2019-03-12,SPINSTERHOOD,ISLAM,SENIOR_HIGH_SCHOOL,,31,71,07,female,32
2,314840386426146816,2019-03-19,MARRIED,ISLAM,ASSOCIATE_BACHELOR_3,,35,78,31,female,25
3,400417947616976896,2019-11-10,MARRIED,ISLAM,SENIOR_HIGH_SCHOOL,googleadwords_int,31,74,07,male,32
4,400421415752728576,2019-11-10,SPINSTERHOOD,ISLAM,SENIOR_HIGH_SCHOOL,kreditpedia,31,75,09,male,27


In [11]:
df4 = load_data_from_pickle(file_path, '设备信息.pkl')
df_device = df4[['customer_id','brand']].copy() 
df_device['brand'] = df_device['brand'].astype(str).str.lower()
df_device.head()

,customer_id,brand
0,145866999474814976,samsung
1,156498181539368960,oppo
2,159786071656632320,vivo
3,161268255131312128,xiaomi
4,163266147299753984,oppo


In [13]:
df5 = load_data_from_pickle(file_path, '盈利性.pkl')
df_profit = df5[['customer_id','last_paid_now','profit_rmb']].copy() 
df_profit.head()

,customer_id,last_paid_now,profit_rmb
0,145866999474814976,235.0,885.25
1,156498181539368960,3.0,4371.10
2,159786071656632320,190.0,1118.95
3,161232718844534784,143.0,1051.25
4,161268255131312128,229.0,524.00


In [26]:
df_all1 = df_baseinfo.merge(df_history,on = 'customer_id',how='left')\
                     .merge(df_device,on = 'customer_id',how='left')\
                     .merge(df_collection,on = 'customer_id',how='left')\
                     .merge(df_profit,on = 'customer_id',how='left')
df_all1['freq'] = df_all1.apply(lambda x: 1 if min(x['last_apply_now'],x['last_paid_now'])<=15 else 0 ,axis = 1)
dummy_list = ['marital_status','religion','education','media_source',	'provincecode',	'citycode',	'districtcode',	'gender','brand']
frame1 = df_all1[['customer_id']].copy()
for cols in dummy_list:
    temp_list = df_all1[cols].value_counts().index.tolist()
    temp_dummy = pd.get_dummies(df_all1[cols])[temp_list]
    temp_dummy.columns = [cols + '_' + str(col) for col in temp_dummy.columns]
    frame1 = frame1.join(temp_dummy)
var_dummy = frame1
var_dummy.head()
df_all11 = df_all1.merge(var_dummy,on = 'customer_id',how = 'left')
save_data_to_pickle(df_all11,file_path,'df_all1.pkl')
df_all1.shape
df_all1.head()


(143044, 25)

,customer_id,reg_date,marital_status,religion,education,media_source,provincecode,citycode,districtcode,gender,age,cnt_loan,cnt_extend,max_extend,cnt_advance,last_apply_now,is_new,brand,max_late_days,rucui_loan,rucui_order,dpd7+,last_paid_now,profit_rmb,freq
0,363324722938687488,2019-07-31,MARRIED,ISLAM,SENIOR_HIGH_SCHOOL,organic,32,73,14,male,42,2,1,1,0,177.0,0,vivo,163.0,2.0,2.0,1.0,167.0,383.50,0
1,312257284621246464,2019-03-12,SPINSTERHOOD,ISLAM,SENIOR_HIGH_SCHOOL,,31,71,07,female,32,2,0,0,0,335.0,0,NaN,328.0,2.0,2.0,1.0,325.0,224.10,0
2,314840386426146816,2019-03-19,MARRIED,ISLAM,ASSOCIATE_BACHELOR_3,,35,78,31,female,25,2,2,1,0,198.0,0,oppo,166.0,2.0,4.0,1.0,166.0,551.75,0
3,400417947616976896,2019-11-10,MARRIED,ISLAM,SENIOR_HIGH_SCHOOL,googleadwords_int,31,74,07,male,32,2,0,0,0,9.0,0,oppo,NaN,0.0,0.0,0.0,33.0,257.10,1
4,400421415752728576,2019-11-10,SPINSTERHOOD,ISLAM,SENIOR_HIGH_SCHOOL,kreditpedia,31,75,09,male,27,2,0,0,1,61.0,0,oppo,47.0,1.0,1.0,1.0,46.0,398.00,0


## APP

In [ ]:
APP计算APP的频数字典

In [10]:
time1 = time.time()
df5 = load_data_from_pickle(file_path, 'APP信息.pkl')
df51 = df5[~df5['packages_cleaned_2'].isna()]
df51 = df51[df51['packages_cleaned_2'] != '']
df_app = dp.from_json(df51, 'packages_cleaned_2')
df_app = df_app[['customer_id','appName']]
df_app.head()
save_data_to_pickle(df_app,file_path,'df_app.pkl')
time2 = time.time() 
print('run_time: ' ,time2-time1)

new columns done
packageName
versionName
versionCode
temp
appName
installTime
run time = 58.973392963409424


,customer_id,appName
0,145866999474814976,CM Lite
1,145866999474814976,WhatsApp
2,145866999474814976,PicsArt
3,145866999474814976,LINE
4,145866999474814976,UKU


run_time:  115.30698680877686


In [3]:
app_data1 = load_data_from_pickle(file_path,'df_app.pkl')
time1 = time.time()
print(time1)
high_cut = 0.4 
mid_cut = 0.3
app_freq = pd.DataFrame(app_data1['appName'].value_counts()).reset_index()
print('训练集的不同app数量: ' , app_freq.shape[0])
app_freq.rename(columns={'index': 'app', 'appName': 'freq'}, inplace=True)
cnt_loan = app_data1['customer_id'].nunique()
app_freq['rate_freq'] = app_freq['freq'].apply(lambda x: x/cnt_loan)
app_freq['high_freq_app'] = app_freq['rate_freq'].apply(lambda x: 1 if x >= float('%f'%high_cut) else 0)
print('高频APP个数', app_freq['high_freq_app'].sum())
app_freq['mid_freq_app'] = app_freq['rate_freq'].apply(lambda x: 1 if (float('%f'%mid_cut)<= x <float('%f'%high_cut)) else 0)
print('中频APP个数', app_freq['mid_freq_app'].sum())
app_freq['low_freq_app'] = app_freq['freq'].apply(lambda x: 1 if x <= 1 else 0)
print('低频APP个数(仅一人安装)', app_freq['low_freq_app'].sum())
app_freq.shape
save_data_to_pickle(app_freq,file_path,'app_freq.pkl')
app_freq.to_csv(file_path  + 'app_freq_train_dict.csv', index=False)
time2 = time.time() 
print('run_time: ' ,time2-time1)

1583680439.658638
训练集的不同app数量:  122963
高频APP个数 28
中频APP个数 24
低频APP个数(仅一人安装) 49745


(122963, 6)

run_time:  2318.9004859924316


In [3]:
time1 = time.time()
df_app = load_data_from_pickle(file_path,'df_app.pkl')
app_freq = pd.read_csv(file_path  + 'app_freq_train_dict.csv')
dict1 = dict(zip(app_freq['app'],app_freq['rate_freq']))
app_list = [k for k,v in dict1.items() if v>=0.1]
len(app_list)
df_app1 = df_app[df_app['appName'].isin(app_list)]
df_app1.shape
frame1 = pd.crosstab(df_app1['customer_id'],df_app1['appName'])
frame1.shape
save_data_to_pickle(frame1,file_path,'var_app.pkl')
frame1.head()
time2 = time.time()
print('run_time: ', time2 - time1)

176

(5917483, 2)

(126954, 176)

appName,AKULAKU,AdaKami,AdaPundi,Agen Masukan Market,AgenMasukanMarket,Akulaku,Analytics,Android Accessibility Suite,Android Easter Egg,Android System WebView,AndroidAccessibilitySuite,AndroidEasterEgg,AndroidSystemWebView,Atci_service,AudioEffect,BABE,BCA mobile,BPJSTKU,Blibli.com,Briefing,Browser,Browserplug,Bukalapak,Cairin,Cashcepat,Cashwagon,Catatan,Catatan Keep,Chrome,ConfigUpdater,Cuaca,DANA,DanaRupiah,Do-It,Download,Drive,Duo,Easycash,Email,Excel,Facebook,Facebook App Installer,Facebook App Manager,Facebook Services,FacebookAppInstaller,FacebookAppManager,FacebookServices,Finmas,Foto,Foundation,GOJEK,Galeri,Gboard,GetApps,Gmail,Gojek,Google,Google Backup Transport,Google One Time Init,Google Play Film,Google Play Game,Google Play Musik,Google Play Store,Google Play services for Instant Apps,GoogleOneTimeInit,GooglePlayFilm,GooglePlayGame,GooglePlayMusik,GooglePlayStore,Grab,Hago,Hangouts,Indodana,Instagram,JOOX,JULO,Jam,Ju0026T Express,KTA KILAT,Kalender,Kalkulator,Kamera,Kerangka Kerja Layanan Google,KerangkaKerjaLayananGoogle,Kompas,Kontak,Kredinesia,Kredit Pintar,KreditPintar,KreditQ,Kredivo,LINE,Layanan Google Play,Layanan Transfer Backup Google,LayananGooglePlay,LayananTransferBackupGoogle,Lazada,Lens,LinkedIn,Lite,Maps,Mesin Google Text-to-speech,Mesin yang beragam,MesinGoogleText-to-speech,Messenger,Mi Remote,MiWebView,Mobile Legends: Bang Bang,Modal Nasional,Musik,My Home Credit Indonesia,MyTelkomsel,OVO,OneDrive,Pembayaran aman,Pembersih,Pemindai,Pengelola Akun Google,Penyiapan Android,Penyiapan Mitra Google,PenyiapanAndroid,PenyiapanMitraGoogle,Perekam,Perekam Suara,Pesan Sistem,PhotoGrid,Photos,Pinjam Yuk,PinjamDuit,PinjamanGo,PowerPoint,Radio,Rupiah Cepat,RupiahCepat,S LIME,SHAREit,Samsung Internet,Samsung Push Service,ShareMe,Shelper,Shopee,Shopintar,Siaran sel,Sinkronisasi Google Kalender,Sinkronisasi Google Kontak,SinkronisasiGoogleKalender,SinkronisasiGoogleKontak,Skype,Spotify,Telegram,Tema,TikTok,Tokopedia,TouchPal Englishgb Pack,TouchPal Indonesian Pack,TouchPal Pinyin/Bihua Pack,TouchPal SkinPack Default White,Traveloka,TunaiKita,Tunaiku,Twitter,UC Browser,UKU,UangMe,Unduhan,V-Appstore,WPS Office,WPSOffice,WhatsApp,Wizard Penyiapan,Word,YouTube,com.mfashiongallery.emag,maucash,msa,org.codeaurora.bluetooth
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
145866999474814976,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
156498181539368960,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0
161268255131312128,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0
163266147299753984,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,1,1,1,0,0,1,0,0,1,1,1,0,0,1,1,1,1,0,0,0,0,2,0,0,0,1,0,1,0,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,2,0,1,0,0,0,0,1,1,0,0,1,1,0,0,1,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,1,1,0,1,1,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,1,1,1,1,0,1,0,0,0,1,1,0,0,1,0,1,0,0,1,0,0,0,0
164066402572308480,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,

run_time:  1240.4591670036316
